# MLFlow demo

Start by watching this <3 min explanation
https://www.youtube.com/watch?v=8SfZ1ElgpdU

In [1]:
import xgboost as xgb
import pandas as pd
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from sklearn.feature_extraction import DictVectorizer

from sklearn.metrics import mean_squared_error
import pickle
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment_xgboost")

<Experiment: artifact_location='/Users/joaorochaemelo/code/IH/aieng/ai-engineering-bootcamp/course/week4_update/day1/mlruns/1', creation_time=1723567415967, experiment_id='1', last_update_time=1723567415967, lifecycle_stage='active', name='nyc-taxi-experiment_xgboost', tags={}>

In [2]:
# Preprocessing Data
# function to read the data and remove outliers
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
# preprocessing functions for train, validation dataset
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


### Train XGBOOST and track experiment with hyperparameter tunning manually

In [4]:
# change dataset format to DMatrix as per XGBOOST required format
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [5]:
# define objective function to use in hyperopt method fmin
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        model = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=3
        )
        y_pred = model.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [6]:
# hyper parameter tunning and logging information from each iteration to mlflow
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 10, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

[0]	validation-rmse:9.81468                          
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:48:04] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:8.42658                          
[2]	validation-rmse:7.66657                          
[3]	validation-rmse:7.25799                          
[4]	validation-rmse:7.03999                          
[5]	validation-rmse:6.91874                          
[6]	validation-rmse:6.84895                          
[7]	validation-rmse:6.80680                          
[8]	validation-rmse:6.78031                          
[9]	validation-rmse:6.76482                          
[0]	validation-rmse:6.88375                                                   
 20%|██        | 1/5 [00:01<00:04,  1.15s/trial, best loss: 6.764823500929617]

/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:48:05] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:6.77237                                                   
[2]	validation-rmse:6.75758                                                   
[3]	validation-rmse:6.74955                                                   
[4]	validation-rmse:6.73661                                                   
[5]	validation-rmse:6.73029                                                   
[6]	validation-rmse:6.72266                                                   
[7]	validation-rmse:6.71618                                                   
[8]	validation-rmse:6.71461                                                   
[9]	validation-rmse:6.71157                                                   
[0]	validation-rmse:11.46938                                                  
 40%|████      | 2/5 [00:02<00:03,  1.16s/trial, best loss: 6.711565763334313]

...

/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:48:07] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.08905                                                   
[1]	validation-rmse:6.75192                                                   
[2]	validation-rmse:6.71175                                                   
[3]	validation-rmse:6.70304                                                   
[4]	validation-rmse:6.69340                                                   
[5]	validation-rmse:6.68293                                                   
[6]	validation-rmse:6.67896                                                   
[7]	validation-rmse:6.67413                                                   
[8]	validation-rmse:6.66800                                                   
[9]	validation-rmse:6.66068                                                   
 80%|████████  | 4/5 [00:04<00:01,  1.08s/trial, best loss: 6.660680704413644]

/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:48:08] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.84549                                                   
[1]	validation-rmse:8.45270                                                   
[2]	validation-rmse:7.66632                                                   
[3]	validation-rmse:7.23607                                                   
[4]	validation-rmse:6.99691                                                   
[5]	validation-rmse:6.86398                                                   
[6]	validation-rmse:6.78835                                                   
[7]	validation-rmse:6.74265                                                   
[8]	validation-rmse:6.71447                                                   
[9]	validation-rmse:6.69216                                                   
100%|██████████| 5/5 [00:06<00:00,  1.23s/trial, best loss: 6.660680704413644]


/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



### Train XGBOOST and track using mlflow autolog


In [7]:
mlflow.xgboost.autolog(disable=True)


In [9]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    # obtained from previous run for hyperparameter tuning
    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    model = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = model.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(model, artifact_path="models_mlflow")

/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:51:21] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/joaorochaemelo/code/IH/venv_ironhack/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:53:26] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
